```
streamlit run your_script.py [-- script args]
```

In [52]:
%%writefile offozaynapp/main.py

import streamlit as st
import pandas as pd
import numpy as np

st.title('Offshore Leaks')


DATE_COLUMN = 'date/time'
DATA_URL = ('https://s3-us-west-2.amazonaws.com/'
         'streamlit-demo-data/uber-raw-data-sep14.csv.gz')

@st.cache
def load_data(nrows):
    data = pd.read_csv(DATA_URL, nrows=nrows)
    lowercase = lambda x: str(x).lower()
    data.rename(lowercase, axis='columns', inplace=True)
    data[DATE_COLUMN] = pd.to_datetime(data[DATE_COLUMN])
    return data


# Create a text element and let the reader know the data is loading.
data_load_state = st.text('Loading data...')
# Load 10,000 rows of data into the dataframe.
data = load_data(10000)
# Notify the reader that the data was successfully loaded.
#data_load_state.text('Loading data...done!')
data_load_state.text("Done! (using st.cache)")

if st.checkbox('Show raw data'):
    st.subheader('Raw data')
    st.write(data.head())

st.subheader('Number of pickups by hour')

hist_values = np.histogram(
    data[DATE_COLUMN].dt.hour, bins=24, range=(0,24))[0]

st.write(hist_values)

st.bar_chart(hist_values)

st.subheader('Map of all pickups')

st.map(data)

hour_to_filter = 17
filtered_data = data[data[DATE_COLUMN].dt.hour == hour_to_filter]
st.subheader(f'Map of all pickups at {hour_to_filter}:00')
st.map(filtered_data)

hour_to_filter = st.slider('hour', 0, 23, 17)  # min: 0h, max: 23h, default: 17h

Overwriting offozaynapp/main.py


In [55]:
%%writefile offozaynapp/main.py

import streamlit as st
import pandas as pd
import numpy as np

st.title('Offshore Leaks')


DATA_LAT_LON = ('https://drive.google.com/file/d/10l-06L2waxvG6pKbCwuX1wtPtrPjgPO4/view?usp=sharing')

JURISDICTION_DATA_URL = ('https://drive.google.com/file/d/10eXidO511doY2M7dLQMZE00lGSVu_Vez/view?usp=sharing')

# Taken with modification from
# https://newbedev.com/pandas-how-to-read-csv-file-from-google-drive-public"""
get_url = lambda u: 'https://drive.google.com/uc?export=download&id=' + u.split('/')[-2]
    
    

@st.cache
def load_data(url, nrows):
    data = pd.read_csv(get_url(url), nrows=nrows) 
    return data


# Create a text element and let the reader know the data is loading.
data_load_state = st.text('Loading data...')
# Load 10,000 rows of data into the dataframe.
data = load_data(JURISDICTION_DATA_URL, 10000)
# Notify the reader that the data was successfully loaded.
# data_load_state.text('Loading data...done!')
data_load_state.text("Done! (using st.cache)")

data_load_state = st.text('Loading data...')
# Load 10,000 rows of data into the dataframe.
lat_lon_data = load_data(DATA_LAT_LON, 10000)
# Notify the reader that the data was successfully loaded.
# data_load_state.text('Loading data...done!')
data_load_state.text("Done! (using st.cache)")

if st.checkbox('Show raw data'):
    st.subheader('Raw data')
    st.write(data.head())
    st.subheader('Jurisdiction Source')
    hist_values = data['jurisdiction_source'].value_counts().sort_values()
    st.bar_chart(hist_values)
    
if st.checkbox('Show Latitude & Longitude data'):
    st.subheader('Raw data')
    st.write(lat_lon_data.head())
    st.map(lat_lon_data)


Overwriting offozaynapp/main.py


In [27]:
import pandas as pd


orig_url = 'https://drive.google.com/file/d/10eXidO511doY2M7dLQMZE00lGSVu_Vez/view?usp=sharing'
file_id = orig_url.split('/')[-2]
dwn_url='https://drive.google.com/uc?export=download&id=' + file_id

data = pd.read_csv(dwn_url)
data.head()

,jurisdiction_source,jurisdiction_target,weight
0,BERMU,XXX,1
1,BVI,BVI,156
2,BVI,COOK,10
3,BVI,LABUA,5
4,BVI,MALAY,4


In [31]:
import numpy as np
hist_values = np.histogram(
    data[DATE_COLUMN].dt.hour, bins=24, range=(0,24))[0]

st.bar_chart(hist_values)

NameError: name 'DATE_COLUMN' is not defined

In [30]:
data['jurisdiction_source'].value_counts()

BVI      6
COOK     5
SAM      4
SGP      4
XXX      3
BERMU    1
CAYMN    1
HK       1
LABUA    1
MALAY    1
MAURI    1
STLU     1
Name: jurisdiction_source, dtype: int64